In [8]:
cd drive/MyDrive/TML/A4

/content/drive/MyDrive/TML/A4


In [16]:
!pip install torch torchvision



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [17]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 10.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.5-py3-none-any.whl size=44284 sha256=3b53531e7a43e26b47da8ad1716e5aeeaa0056edffcbe0a1a3f7cd70170dbf86
  Stored in directory: /root/.cache/pip/wheels/bc/52/78/893c3b94279ef238f43a9e89608af648de401b96415bebbd1f
Successfully built grad-cam


In [1]:
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

model = models.resnet50(pretrained=True)
model.eval()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 35.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [2]:
 preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image).unsqueeze(0)  # add batch dimension
    return image, input_tensor


In [3]:
target_layers = [model.layer4[-1]]


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def apply_cam_gpu(method, input_tensor, rgb_img, target_category):
    # Move input tensor to the device
    input_tensor = input_tensor.to(device)
    cam = method(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(target_category)])
    grayscale_cam = grayscale_cam[0, :]
    visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
    return visualization

In [23]:
from pytorch_grad_cam import GradCAM, ScoreCAM, AblationCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

# def apply_cam(method, input_tensor, rgb_img, target_category):
#     cam = method(model=model, target_layers=target_layers)
#     grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(target_category)])
#     grayscale_cam = grayscale_cam[0, :]
#     visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)
#     return visualization

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import os
def visualize_and_save_all_methods(image_path, output_folder):
    original_img, input_tensor = load_image(image_path)
    # Resize the original image to match the input tensor size
    original_img_resized = original_img.resize((224, 224))
    input_image_np = np.array(original_img_resized).astype(np.float32) / 255.0
    class_id = get_prediction(input_tensor)
    gradcam_result = apply_cam_gpu(GradCAM, input_tensor, input_image_np, class_id)
    scorecam_result = apply_cam_gpu(ScoreCAM, input_tensor, input_image_np, class_id)
    ablationcam_result =apply_cam_gpu(AblationCAM, input_tensor, input_image_np, class_id)
    titles = ['Grad-CAM', 'ScoreCAM', 'AblationCAM']
    results = [gradcam_result, scorecam_result, ablationcam_result]
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    plt.figure(figsize=(12, 4))
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.imshow(results[i])
        plt.title(titles[i])
        plt.axis('off')
    plt.suptitle(f"Visualizations for class ID: {class_id}")
    # Save the figure
    image_name = os.path.basename(image_path).split('.')[0]
    output_path = os.path.join(output_folder, f'{image_name}_vis_{class_id}.png')
    plt.savefig(output_path)
    plt.close() # Close the plot to free memory

In [25]:
# !pip install grad-cam

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def get_prediction(input_tensor):
    # Move input tensor to the device
    input_tensor = input_tensor.to(device)
    outputs = model(input_tensor)
    _, predicted = outputs.max(1)
    return predicted.item()

def visualize_all_methods(image_path):
    original_img, input_tensor = load_image(image_path)
    # Resize the original image to match the input tensor size
    original_img_resized = original_img.resize((224, 224))
    input_image_np = np.array(original_img_resized).astype(np.float32) / 255.0

    class_id = get_prediction(input_tensor)

    gradcam_result = apply_cam(GradCAM, input_tensor, input_image_np, class_id)
    scorecam_result = apply_cam(ScoreCAM, input_tensor, input_image_np, class_id)
    ablationcam_result = apply_cam(AblationCAM, input_tensor, input_image_np, class_id)

    titles = ['Grad-CAM', 'ScoreCAM', 'AblationCAM']
    results = [gradcam_result, scorecam_result, ablationcam_result]

    plt.figure(figsize=(12, 4))
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.imshow(results[i])
        plt.title(titles[i])
        plt.axis('off')
    plt.suptitle(f"Visualizations for class ID: {class_id}")
    plt.show()

In [26]:
import os

def main():
    image_folder = "10_images"  # replace with your folder path
    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in sorted(image_paths):
        print(f"\nProcessing: {os.path.basename(image_path)}")
        visualize_and_save_all_methods(image_path, 'output_t2')


if __name__ == "__main__":
    main()


Processing: n01443537_goldfish.JPEG


100%|██████████| 64/64 [00:05<00:00, 10.86it/s]



Processing: n01491361_tiger_shark.JPEG


100%|██████████| 64/64 [00:06<00:00, 10.46it/s]



Processing: n01608432_kite.JPEG


...

100%|██████████| 64/64 [00:05<00:00, 10.99it/s]



Processing: n04037443_racer.JPEG


100%|██████████| 64/64 [00:05<00:00, 10.99it/s]



Processing: n07747607_orange.JPEG


100%|██████████| 64/64 [00:05<00:00, 10.96it/s]
